# The Achilles' heel of Stochastic Gradient Descent (SGD)

In this note we discuss the **main limitation, the Achilles' heel**, of the Stochastic Gradient Descent (SGD) optimization algorithm in the context of training Deep Artificial Neural Networks (DANNs) or Deep Neural Networks (DNNs). Then, a bag of techniques to overcome this limitation is presented. 

We will consider only the mini-batch SGD and limit the discussion in the context of DNNs.


SGD is a simple iterative algorithm used to find suitable weights of the neurons in the hidden layers of ANN by reducing a cost function. There are two aspects of its performance that concerns us.

- Effectiveness: find the optimal (or near-optimal) solution that incurs the lowest error or cost
- Efficiency: achieve effectiveness using less resource such as smaller number of iterations and smaller memory

Ensuring both efficiency and effectiveness of SGD in training DNNs is challenging due to intrinsic and extrinsic factors. Both contribute to the making of **the Achilles' heel** of SGD. While the intrinsic factor is driven by the algorithmic nature of SGD, the extrinsic factor is caused by the parameter space of the DNN cost function. We will discuss about these two factors and techniques to overcome the challenges caused by these factors. Of these two, we will see that the extrinsic factor, i.e., the cost space would demand more emphasis. We will use calculus profusely to navigate through the cost space. But the choice of any mathematical tools need to be first driven by insights and physical intuitions. Thus, prior using calculus based techniques, we would use words to map our imagination. More precisely, we would use **metaphors** as handles to the complex and nuanced reality of the cost space. But we have to take precaution! 


## Power and Peril of Metaphors

The high-dimensional cost space of DNN is a land of mystery. Because it extends the three-dimensional reality of our everyday life. We connect these two worlds through metaphors. For example, we use the metaphor of mountain to represent the overall cost space. Training of a DNN in search of a solution is conveyed aptly by the metaphor of a downhill walk from the top of the mountain. An optimal solution that SGD tries to reach at is best represented by the metaphor of reaching the lowest valley at the foot of the mountain. The fact that the downhill walk of SGD is not smooth and can be stalled due to various characteristics of the cost space can be grasped easily by using metaphors of plateau, saddle point, ravine, canyon, cliff, etc. We will see that metaphors are extremely powerful for selecting suitable mathematical techniques. But is a peril in the use of metaphors as well.

Sometimes metaphors are misrepresentations of the high-dimensional reality of DNNs. Even if we are lucky then metaphors are at best poor representations of the cost space. For example, when we use the metaphor of a plateau to describe a location in the cost space we have no idea how a plateau would look like in 1,000,000 dimension. A cliff or a saddle point in that high dimension fails our imagination. Still we use these metaphors to guide our understanding, as a clue to dive into the unseen world and develop useful intuitions. These metaphors bear strong gravity towards our three-dimensional reality, which we compensate with mathematical machinary in the high-dimensional cost space of DNNs.

In this note, we will use examples of only two and three-dimensional cost functions, for the convenience of understanding. Thus, in the lower-dimensional cost space of our examples the metaphors could be somewhat meaningful. But as we depart to the mystery world of very high-dimension, they will loose the touch of the ground. Thus we should constantly remind ourselves that in higher-dimensional cost space of DNNs, metaphors as used in their three-dimensional analogues might betray us. We must not let our unconscious mind to be slack in discounting the metaphors while transitioning to the higher-dimensional reality of DNN. 


<img src="http://engineering.unl.edu/images/uploads/SGD-Metaphor.png" width=600 height=300>



Now let's begin the discussion on the Achilles' heel of SGD. First, we will describe the ANN training process briefly.


### Training an ANN

In training an ANN using a set of labeled data $X$, we update a set of parameters to optimize an objective function. The parameters are the weights and biases of the neurons in the hidden layers. Using vector notation we denote them together with $\vec{w}$. The objective function is the network loss or cost function:

$J(\vec{w}) = \frac{1}{\mid X \mid}\sum_{\vec{x} \in X}J(\vec{x}, \vec{w})$

Typically $j$ is the sum of a classification loss (e.g., cross-entropy) and a regularization loss on $\vec{w}$. The training goal of an ANN is to minimize the cost by finding suitable weights $\vec{w}$.

But minimization of the ANN cost function $J$ is non-trivial. Because $J$ is **non-convex** due to its nonlinear dependence on the weights and bias parameters. Thus, unfortunately, $J$ does not admit a closed-form solution. This is where SGD kicks in. It offers an approximate solution.



## SGD: a Quick Premier

SGD optimization algorithm is based on the Stochastic Approximation method invented in 1951 by Robbins and Monro: https://projecteuclid.org/download/pdf_1/euclid.aoms/1177729586 


SGD minimizes $J$ in an iteraive fashion. At each iteration SGD uses a random a data point (or a mini-batch of data points in SGD mini-batch) at a time to update the weight (and bias) parameters $\vec{w}$ of the neurons in the hidden layers. It does so by finding the steepest direction in the weight space, which is given by the gradient of $J$ with respect to $\vec{w}$:  $\nabla_w J(\vec{w})$


First, it computes the gradient of $J(\vec{w})$. Then, it tries to minimize the cost by updating the weights $\vec{w}$ by directly subtracting the gradient of the cost function ($\nabla_w J(\vec{w})$) multiplied by the learning rate $\eta$, which is a **scalar constant**. The SGD weight update rule is as follows:

- $\vec{w}_{t+1} = \vec{w}_t - \eta \nabla_w J(\vec{w}_t)$

Essetially SGD computes the cost function derivative at the present location in the cost function space. This tells what the slope is and which way is down. Then, it takes a small step down and repeat until it gets to the minimum value of $J(\vec{w})$. The size of a step is set by the positive scalar constant learning rate $\eta$. Note that this step size is same in all dimensions (for all weight parameters). Later we will see that SGD converges faster if each dimension has its own learning rate. In that case the learning rate would be a diagonal matrix of dimension $d x d$, where $d$ is the number of weight parameters.

Let's visualize the cost space below for a simple two-dimensional cost function. Here, x and y represent the two weights and z represents the cost function:
$z = x^2 - x*y$

<img src="http://engineering.unl.edu/images/uploads//SGD-2D.png" width=600 height=300>

We can use **metaphors** to gain an intuitive understanding of how SGD minimizes $J$.

The cost function is a "mountain". SGD tries to get to the "valley" of the mountain at which cost is zero. Using the weight update rule, SGD "walks down the mountain" by finding "steepest slopes" at each to reach the lowest point in the "valley". 

<img src="http://engineering.unl.edu/images/uploads/SGD-MountainMetaphor.png" width=600 height=300>

Note that the SGD cost function is non-convex because of its nonlinear dependence on the weights and bias parameters. Using metaphor again, the cost function is not a single mountain, it could be a mountain range and there are many valleys. As a consequence there is no guarantee that SGD would reach the global minimum point of the cost function. 

<img src="http://engineering.unl.edu/images/uploads/SGD-MountainRangeMetaphor.png" width=600 height=300>

In high-dimensional weight space (i.e., in DNNs) it is not possible to know whether a global minimum exists. Finding the global minimum is a NP-hard problem. Thus, the goal of SGD in training DNNs is to find **a minimum** that generalizes well enough, yielding a test accuracy compatible with our problem. We settle for less, and that's ok!




### SGD: Advantages

There are at least three advantages of SGD.

- First, it is much faster than the batch GD. Because it has small data (a single data or mini-batch) to manipulate at every iteration. 

- Second, it also makes it possible to train on huge training sets, since only one instance (or a small subset of instances) needs to be in memory at each iteration. 

- Third, due to its inherent randomness, SGD is useful to escape from sub-optimal minima.


### SGD: Disadvantage

        SGD has an Achilles' heel. It's sloth! 


SGD (or mini-batch SGD) takes much longer time to converge near the minima of the cost function, as compared to batch GD.


<img src="http://engineering.unl.edu/images/uploads/GradientDescent-Paths.png" width=600 height=300>



There are two main contributing factors to its slow convergence.
- Nature of SGD algorithm
- Topology of the cost function

We describe these two causes.

### Nature of SGD Algorithm

In SGD, at each update of the weight parameters, successive gradient values may be so different that **large oscillations** may occur. Due to its stochastic (i.e., random) nature, SGD is much less regular than Batch GD. Since SGD uses small batches of data, the gradients of the cost function is "noisy". This haapens even on a smooth terrain. Thus, instead of gently decreasing until it reaches the minimum, the cost function will oscillate, decreasing only on average. Over time it will end up very close to the minimum, but once it gets there it will continue to bounce around around the minimum, never settling down. Oscillation is the **evil**!

<img src="http://engineering.unl.edu/images/uploads/SGD-No_Momentum.png" width=800 height=400>


#### Root Cause of the Evil
Let's try to understand the root cause of the oscillations in SGD algorithm. Recall that at each iteration SGD measures the gradient of $J(\vec{w})$ w.r.t all components of $\vec{w}$. In other words, it measures the slopes along all dimensions. Then, it takes a step along the steepest direction. The step size is determined by the scalar constant learning rate hyperparameter $\eta$. This hyperparameter is the root cause of the oscillatory evil!

Typically $\eta$ is chosen by hand. The problem is:
- A too large $\eta$ overshoots the step and SGD movement diverges.
- A too small $\eta$ results in slow learning. 


<img src="http://engineering.unl.edu/images/uploads/SGD-LearningRate.png" width=800 height=400>

Then, how do we find the optimal $\eta$? Can we analytically determine it? 

It appears that we can actually calculate the optimal $\eta$. Let's do this.


### Analytically Find the Optimal $\eta$

First, we expand the cost function $J(\vec{w})$ in a Taylor series about the current weight $\vec{w}_c$:

$J(\vec{w}) = J(\vec{w}_c) + (\vec{w} - \vec{w}_c) \frac{\partial J(\vec{w}_c)}{\partial \vec{w}} + \frac{1}{2}(\vec{w} - \vec{w}_c)^2 \frac{\partial^2 J(\vec{w}_c)}{\partial \vec{w}^2} + ...$

Differentiating both sides w.r.t $\vec{w}$ and ignoring the higher order terms:


$\frac{\partial J(\vec{w})}{\partial \vec{w}}  = \frac{\partial J(\vec{w}_c)}{\partial \vec{w}} +  (\vec{w} - \vec{w}_c) \frac{\partial^2 J(\vec{w}_c)}{\partial \vec{w}^2}$

At the minimum: $\frac{\partial J(\vec{w})}{\partial \vec{w}} = 0$

Thus we get:

$\frac{\partial J(\vec{w}_c)}{\partial \vec{w}} +  (\vec{w} - \vec{w}_c) \frac{\partial^2 J(\vec{w}_c)}{\partial \vec{w}^2} = 0$

=> $\vec{w}\frac{\partial^2 J(\vec{w}_c)}{\partial \vec{w}^2} = \vec{w}_c\frac{\partial^2 J(\vec{w}_c)}{\partial \vec{w}^2} - \frac{\partial J(\vec{w}_c)}{\partial \vec{w}}$

=> $\vec{w} = \vec{w}_c - (\frac{\partial^2 J(\vec{w}_c)}{\partial \vec{w}^2})^{-1}\frac{\partial J(\vec{w}_c)}{\partial \vec{w}} $

Now compare the above equation with the SGD update rule:
Consider the SGD update rule.

$\vec{w} \leftarrow \vec{w} - \eta \nabla_w J(\vec{w})$

Thus, we see that the optimal learning rate is given by:

$\eta_{opt} = (\frac{\partial^2 J(\vec{w}_c)}{\partial \vec{w}^2})^{-1}$

Observe that the $\eta_{opt}$ is no more a scalar constant. Instead it is a $d x d$ matrix. It is computed by taking the inverse of the second-order derivative of the cost function w.r.t. the weight parameters. It is called the **Hessian**. More precisely, the components of the Hessian are given by:

$H_{ij} \equiv \frac{\partial^2 J(\vec{w}_c)}{\partial \vec{w}_i \partial \vec{w}_j}$

with $1 \leq i, j \leq d$.

We have computed $\eta_{opt}$ by ignoring the higher-order terms in the Taylor series expansion of the cost function. Thus, $\eta_{opt}$ does not give the exact optimal learning rate. It may take multiple iterations to locate the minimum even when using $\eta_{opt}$, however, convergence can still be quite fast.

$H$ is a measure of the curvature of $J(\vec{w})$. We will elaborate on curvature later. In two dimensions, the lines of constant $J(\vec{w})$ for a quadratic cost function are oval in shape as shown below. The eigenvectors of $H$ point in the directions of the major and minor axes. The eigenvalues measure the steepness of $J(\vec{w})$ along the corresponding eigendirection.

<img src="http://engineering.unl.edu/images/uploads/SGD-Hessian-Eigenvectors.png" width=800 height=400>


Now that we have found the optimal $\eta$, we realize that in DNNs it's not a scalar constant. The problem is that the calculation of the matrix containing optimal learning rates (i.e., the Hessian) would incur additional cost. We will perform a cost-benefit analysis of this approach later.

But for now we must acknowledge that in DNNs the optimal scalar constant $\eta$ does not exist. It appears that determining either a good scalar constant $\eta$ or the $\eta$ for each dimension (i.e., Hessian) is more of an art than science.

Let's invent that art...

 

#### Art of Killing the Evil

There are at least two approaches to kill the evil of oscillation by finding a suitable $\eta$.

The first approach attempts to find a scalar constant $\eta$, while the second approach presents feasible techniques for finding the optimal $d x d$ matrix of $\eta$, the glorious Hessian! 

- Approach 1: Taming of the Shrew or Simulated Annealing

As SGD nears a minima in the cost surface, we could reduce the oscillations by slowing down the weight updates by decreasing $\eta$. 

This has to be done gently. SGD starts out by taking large steps (i.e., large $\eta$) so that it can make quick progress and escape local minima. Then, we reduce $\eta$ to make the steps smaller and smaller. It will allow the algorithm to settle at the global minimum. This process is known as simulated annealing, inspired from the process in metallurgy of annealing, where molten metal is slowly cooled down.

Simulated annealing can be done by creating a **learning schedule**. The schedule automatically anneals the learning rate based on how many epochs through the data have been done. 

See the following notebook for the definition, implementation and experimentation with various learning schedules:

https://github.com/rhasanbd/Smooth-Taste-of-Keras--Journey-Through-MLPs/blob/master/Keras-MLP%20Classifier-3-Learning%20Rate%20Experiments.ipynb


There are two limitations of using learning schedules with SGD.

- The biggest limitation is that we need to define additional hyperparameters in advance. The optimal settings depend on the model architecture and the data. Tuning of these hyperparameters adds more cost to the training.

- The same learning rate is applied to all weight parameter updates. If we have sparse data, we may want to update the parameters in different extent instead.


A superior approach would be to have separate lerning rate for each dimension, which we can determine by using second-order SGD (i.e., by Hessian). Let's see how it works. This would be our approach 2 to kill the evil.

- Approach 2: Shoot in the Foot of Scalar Constant Learning Rate - Dial Second-Order Derivatives

We can modify the SGD update rule by replacing $\eta$ by the optimal learning rate matrix, which is the inverse of the second-order derivatives of $J\vec{w}$:

- $\vec{w} \leftarrow \vec{w} - H^{-1}g$

Where $g = \nabla_w J(\vec{w})$ is the gradient vector of the cost function $J\vec{w}$ and $H = \nabla_w^2 J(\vec{w})$ is the **Hessian** matrix of second-order derivatives of $J\vec{w}$. 

The above update rule can also be derived using the **Newton method** (more precisely by using the Newton-Raphson method). This method is used to find the minimum of real-valued function through an iterative process of approximation.

We leave aside the derivation of the Newton method based update rule, instead offer an intuitive justification.

The key idea in this update rule is to use the second-order derivatives or Hessian, which provides the **curvature** of the cost space. SGD can move faster by using the curvature of the space into account. Here curvature (given by Hessian) acts as a proxy of the learning rate for each dimension. Consider the following figure. At point "a", since the slope is steep, we would like to increase the learning rate. On the other hand, at point "b", since it's an almost flat region, we would want to reduce the learning rate. But the problem is we don't know exactly how steep or flat these regions are. The gradient of the cost function only tells us the direction of the steepest descent/ascent. In the first-order (or gradient based) SGD we simply make an emprical guess to determine the size of the step without exactly knowing the steepness or flatness of the ground beneath us. Typically we play conservative to avoid overshooting by using a smaller learning rate than needed. 

<img src="http://engineering.unl.edu/images/uploads/SGD-Curvature.png" width=800 height=400>

The beauty and benefit of the Hessian is that it can quantify the steepness/flatness. For example, in the above figure, the curvature at point "a" is large, while at point "b" is small. Thus, by measuring the curvature of a space at a point we could know how fast/slow we neeed to move towards the minimum. In other words, curvature provides the exact step size suitable for the slope of the ground beneath. The curvature is the "learning rate". 

To offer a little bit more intuition, curvature of a circle is inversely proportional to its radius. Smaller circles bend more sharply, and hence have higher curvature, while larger circles bend gently due to smaller curvature.


<img src="http://engineering.unl.edu/images/uploads/SGD-CurvatureCircles.png" width=600 height=300>


Let's understand how curvature (second-order derivative) might be useful over the first-order SGD.

In the first-order SGD we only use gradient with a hand-tuned step size (learning rate). But we don't know whether the step size is too small or large. Curvature tells us whether a gradient step will cause as much of an improvement as we would expect based on the gradient alone. Following figure shows how different forms of curvature affect the relationship between the value of the cost function predicted by the gradient and the true value.

Here the cost function is quadratic. The dashed line indicates the value of the cost function we would expect based on the gradient information alone as we make a gradient step downhill. We need to understand in which scenarios we should have larger step size and where we should reduce the step size.

- On the far left, curvature is negative. Thus, the cost function actually decreases faster than the gradient predicts. Step size (learning rate) should be larger.

- In the middle figure where the curvature is zero, the gradient predicts the decrease correctly. 

- On the far right, the curvature is positive , hence the function decreases slower than expected and eventually begins to increase. If we set the step size larger then it will overshoot.

Thus, we see that the gradient alone doesn't have the information to determine the step size, which we can get from Hessian.


<img src="http://engineering.unl.edu/images/uploads/SGD-Curvature-GD.png" width=600 height=300>

Following figure shows how the Newton method is applied to SGD. The solid red curve shows the cost function $J$. To find the curvature of this function, at each iteration (at time t), we make a second-order (quadraic) approximation of the cost function (its Hessian) $J_{quad}$, represented by the dotted blue curve. To get towards the minimum of $J_{quad}$ we update $\vec{w}_t$ as follows: 

- $\vec{w_{t+1}} = \vec{w_t} - H^{-1}g$

<img src="http://engineering.unl.edu/images/uploads/SGD-NewtonMethod.png" width=600 height=300>

In the above update rule, Hessian determines the optimal step size along each dimension in the cost space. 



### Some Useful Properties of Hessian Matrix


In Deep Learning, the Hessian matrix is real and symmetric. Thus it permits eigendecomposition, i.e.,  
we can decompose it into a set of real eigenvalues and an orthogonal basis of eigenvectors. The fact that we can eigendecompose Hessian provides some useful properties in the context of SGD.



- Property 1: 

The maximum eigenvalue determines the maximum second derivative and the minimum eigenvalue determines the minimum second derivative. The eigenvalues measure the steepness of the cost function along the corresponding eigendirection.

- Property 2:

If the cost function we minimize can be approximated well by a **quadratic function**, then the eigenvalues of the Hessian determine the scale of the learning rate. The optimal rate for the $i$-th weight $w_i$ would be the inverse of the eigenvalue: 

$\eta_{opt, i} = \frac{1}{\lambda_i}$


Before we derive the above equation, let's ask how justified is our assumption about the cost function. Well, if we consider the high-dimensional cost space, the assumption that the cost function is quadratic near the current location is mostly valid, unless the current point resides on a wide plateau. 

To derive the equation for optimal learning rate we expand $J(\vec{w})$ about a minimum:

$J(\vec{w}) \approx J(\vec{w}_{min}) + \frac{1}{2} (\vec{w} - \vec{w}_{min})^T J(\vec{w}_{min}) (\vec{w} - \vec{w}_{min}) $

Differentiating it w.r.t. $\vec{w}_{min}$:

$\nabla J(\vec{w}) \approx 0 + J(\vec{w}_{min})(\vec{w} - \vec{w}_{min}) $

Now let's plug this in the first-order SGD update rule:

$\vec{w}_{t+1} = \vec{w} - \eta \nabla J(\vec{w})$

=> $\vec{w}_{t+1} = \vec{w} - \eta  H(\vec{w}_{min})(\vec{w} - \vec{w}_{min})$

Subtracting $\vec{w}_{min}$ from both sides gives:

$(\vec{w}_{t+1} - \vec{w}_{min}) = (I - \eta  H(\vec{w}_{min})(\vec{w} - \vec{w}_{min})$

Note that $(I - \eta  H(\vec{w}_{min})$ is diagonal with components $(1 - \eta \lambda_i)$. The updates would converge if for all dimension $i$:

$\mid 1 - \eta \lambda_i \mid < 1$

=> $\eta < \frac{2}{\lambda_i}$


If the eigenvectors of $H$ are not aligned with the weight coordinate axes (as shown below), then we might constrained to have a single scalar learning rate for all weights. In such case we may use the largest eigenvalue of $H$, i.e., $\lambda_{max}$ to determine the scalar learning rate: 

$\eta < \frac{2}{\lambda_{max}}$

For fastest convergence we have:

$\eta_{opt} = \frac{1}{\lambda_{max}}$


<img src="http://engineering.unl.edu/images/uploads/SGD-Hessian-Eigenvectors.png" width=800 height=400>


However, assuming that the weight coordinate axes are aligned with the corresponding eigenvectors, the optimal learning rate for the $i$-th weight $w_i$ would be: 

$\eta_{opt, i} = \frac{1}{\lambda_i}$


- Property 3:

The eigenvalues of the Hessian determine whether the zero gradient point is a local maximum, local minimum, or saddle point. 

- When the Hessian is positive definite (all its eigenvalues are positive), i.e., $\nabla J(\vec{w}) = 0 and \nabla^2 J(\vec{w}) > 0$, the point is a local minimum.

- When the Hessian is negative definite (all its eigenvalues are negative), i.e., $\nabla J(\vec{w}) = 0 and \nabla^2 J(\vec{w}) < 0$, the point is a local maximum.

- When at least one eigenvalue is positive and at least one eigenvalue is negative, we know that x is a local maximum on one cross section of $J(\vec{w})$ but a local minimum on another cross section. It refers to a region in cost space, known as saddle points. We will discuss about saddle points later.


******************************************************************************************


Now that we have removed the root cause of the evil (i.e., the scalar constant learning rate hyperparameter), what prevents us from using the second-order SGD in training DNNs?

Well, for one, it's prohibitively expensive. Let's give the drawbacks of the Newton method in DNN:

- For convergence it requires a good initial estimate of the solution. However, this estimate is not available since the weights are relatively meaningless entity by themselves. 

- Each iteration requires the computation of the Hessian matrix (of size $d^2$ for $d$ number of weights). The space complexity of storing Hessian is $O(d^2)$. Moreover, the time-complexity of inverting the Hessian is $O(d^3)$. Thus, Newton's method is expensive both in terms of memory (storage) and time (computation).

- Finally, it typically requires much larger batch sizes, e.g., 10,000. These large batch sizes are required to minimize fluctuations in the estimates of $H^1g$. Suppose that H is estimated perfectly but has a poor condition number (i.e., the ratio between the max eigenvalue and min eigenvalue is large). Multiplication by $H$ or its inverse amplifies pre-existing errors, in this case, estimation errors in $g$. Very small changes in the estimate of $g$ can thus cause large changes in the update $H^1g$, even if $H$ were estimated perfectly. Of course, $H$ will be estimated only approximately, so the update $H^1g$ will contain even more error than we would
predict from applying a poorly conditioned operation to the estimate of $g$.

To alleviate the second drawback (storage & computation) of the Newton method several **quasi-Newton** methods are invented. These methods iteratively build up an approximation to the Hessian using information gleaned from the gradient vector at each step. One of the best of these variants is the **Broyden-Fletcher-Goldfarb-Shanno (BFGS)** algorithm. The time-complexity of the BFGS algorithm is much less than Newton's method, i.e., $O(d^2)$. 

The space-complexity of the Newton method can be alleviated by using a limited memory BFGS, or L-BFGS, where $H$ is approximated by a diagonal plus low rank matrix.

But it is mainly due to the cost of computation of BFGS/L-BFGS, which is $O(d^2)$, as compared to the $O(d)$ cost of backpropagation, that Newton or Quasi-Newton methods are not feasible to train DNNs.

    Where does it leave us then?

We are in a quandary! We found that the second-order methods are our elixir to slow convergence, they obviate the need of the scalar constant learning rate hyperparameter. But we cannot afford them.

What if we could invent an approximate yet cheap version of Newton method! In 1989, Becker and LeCun proposed a computationally feasible approximation to Newton Method.

http://yann.lecun.com/exdb/publis/pdf/becker-lecun-89.pdf


This approach is known as the **diagonal approximation** to the Hessian approach. In this approach, we simply replace the off-diagonal elements of the Hessian with zeros. As a result, its inverse is trivial to evaluate. The original formulation of this approach is done by Yann LeCunn in 1987. It was given in his PhD dissertation, which was written in French and titled "Modeles connexionnistes de l'apprentissage" (connectionist learning models).
https://nyuscholars.nyu.edu/en/publications/phd-thesis-modeles-connexionnistes-de-lapprentissage-connectionis

Let's see how to compute the Hessian by using the diagonal approximation technique.

Consider the following topology of an ANN. 

<img src="http://engineering.unl.edu/images/uploads/SGD-DiagonalApproximation.png" width=600 height=300>


In this feed-forward ANN, each neuron computes a weighted sum of its inputs of the form:

$a_j = \sum_i w_{ji} z_i$

Here $z_i$ is the activation of a neuron (or input). It sends a connection to neuron $j$, The weight of this connection is $w_{ji}$ (going from layer $i$ to layer $j$). The above weighted sum is transformed by a nonlinear activation function $h(.)$ to give activation $z_j$ of neuron $j$:

$z_j = h(a_j)$

The forward propagation is computed by successively applying the above two equations from the input layer all through to the final layer.

We are interested to evaluate the derivative of the cost function $J(\vec{w})$ w.r.t. a weight neuron $w_{ji}$. Note that $J(\vec{w})$ depends on the weight $w_{ji}$ only via the summed input $a_j$ to neuron $j$. Thus, applying the chain rule for partial derivative, we get:

$\frac{\partial J(\vec{w})}{\partial w_{ji}} = \frac{\partial J(\vec{w})}{\partial a_{j}} \frac{\partial a_j}{\partial w_{ji}}$


Here the first term on the righ-hand side is the error term, denoted by $\delta_k$, and the second term is evaluated as:

$\frac{\partial a_j}{\partial w_{ji}} = \frac{\partial (\sum_i w_{ji} z_i)}{\partial w_{ji}}= z_i$


Thus, the gradient of the cost function is:

$\frac{\partial J(\vec{w})}{\partial w_{ji}} = \frac{\partial J(\vec{w})}{\partial a_{j}} z_i$

From this, we compute the Hessian matrix: $\frac{\partial^2 J(\vec{w})}{\partial w_{ji} \partial w_{lk}}$

Now in the diagonal approximation, we only need to compute the diagonal elements of the Hessian, as follows:


$\frac{\partial^2 J(\vec{w})}{\partial w_{ji}^2} = \frac{\partial^2 J(\vec{w})}{\partial a_{j}^2} z_i^2$


To evaluate the above diagonal approximation of Hessian, we need to compute the first term on the right-hand side, i.e., second-order derivatives of the cost function $J(\vec{w})$ w.r.t. the total input to each neuron $a_j$: $\frac{\partial^2 J(\vec{w})}{\partial a_{j}^2}$

Let's do this.

Fisrt, we compute the first-order derivative of $J$ w.r.t. $a_j$, followed by the second-order derivative.

$\frac{\partial J(\vec{w})}{\partial a_j} = \sum_k \frac{\partial J(\vec{w})}{\partial a_k} \frac{\partial a_k}{\partial a_j}$

=> $\frac{\partial J(\vec{w})}{\partial a_j} = \sum_k \frac{\partial J(\vec{w})}{\partial a_k} \frac{\partial a_k}{\partial z_j} \frac{\partial z_j}{\partial a_j}$

=> $\frac{\partial J(\vec{w})}{\partial a_j} = \sum_k \frac{\partial J(\vec{w})}{\partial a_k} w_{kj} h^{\prime}(a_j)$

=> $\frac{\partial J(\vec{w})}{\partial a_j} = h^{\prime}(a_j)\sum_k w_{kj}\frac{\partial J(\vec{w})}{\partial a_k}  $


Now, we compute the second-order derivatives of the cost function $J(\vec{w})$ w.r.t. the total input to each unit $a_j$, as follows:

$\frac{\partial^2 J(\vec{w})}{\partial a_j^2} = h^{\prime}(a_j)^2 \sum_k \sum_k^{\prime} w_{kj} w_{k^{\prime}j} \frac{\partial^2 J}{\partial a_k \partial a_k^{\prime}} + h^{\prime \prime}(a_j) \sum_k w_{kj} \frac{\partial J}{\partial a_k}$

By neglecting the off-diagonal elements, we obtain the following.

$\frac{\partial^2 J(\vec{w})}{\partial a_j^2} = h^{\prime}(a_j)^2 \sum_k w_{kj}^2 \frac{\partial^2 J}{\partial a_k^2} + h^{\prime \prime}(a_j) \sum_k w_{kj} \frac{\partial J}{\partial a_k}$


For the above derivation (not shown), we used chain rule and product rule of calculus for finding the second-order derivative for the composition of two functions, such as shown in this simple example:

$\frac{d^2 f(g(x))}{dx^2} = f^{\prime}(g(x)) g^{\prime \prime}(x) + (g^{\prime}(x))^2 f^{\prime \prime}(g(x))$


- Chain rule: $\frac{d f(g(x))}{dx} = g^{\prime}(x) f^{\prime}g((x))$

- Product Rule: $\frac{d}{dx}(f(x) g(x)) = g(x) f^{\prime}(x) + f(x) g^{\prime}(x)$


Finally, using the above expressions we give the diagonal approximation of the Hessian:

$\frac{\partial^2 J(\vec{w})}{\partial w_{ji}^2} = \frac{\partial^2 J(\vec{w})}{\partial a_{j}^2} z_i^2$


So far so good! Does this mean that from now on we can use the diagonal approximation of the Hessian in the Newton method, so that we achieve faster convergence in an affordable manner?

Not quite so! In practice the Hessian is typically found to be strongly nondiagonal. Although diagonal approximation can capture much of the curvature information, there could be some large eigenvalues in the true Hessian not accounted for by the diagonal approximation. There could be many regions in the cost surface where small changes in some weights result in very large changes in the gradient. If the diagonal Hessian terms are relatively small in such regions, a descent step computed strictly from these terms and gradient may be catastrophic. 


Thus, these approximations do not guarantee the optimality of the solution.

    Then, why did we go through a lengthy discussion on the second-order and its approximation based approaches?
    
Well, we will see later that some of the fast optimizers will use intuition from the second-order methods. Hessian (or even its diagonal approximation) for sure have knowledge to accelerate convergence by adapting learning rates per dimension. 

If we need to take just one lesson from the discussion on the second-order methods (for later use), it should be this:

- The second-order derivatives (found on the diagonal of the approximate Hessian) of the cost function w.r.t. each weight parameters contain some information (if not complete) about the curvature of the cost space. Therefore, they would be extremely useful for per-dimension adaptation of the learning rate.


So far we have discussed the oscillatory behavior of SGD, due to which it converges slowly; and identified the root cause to be the constant scalar learning rate. Then, we presented some techniques to aleviate this problem.

    Next we discuss other causes of slow convergence. This time the causes are extrinsic to SGD, contributed by the cost function space.




#### Topology of the Cost Function


The topology of the SGD cost function is also responsible for the slow speed of its convergence. The topological factors vary across from low-dimensional (shallow ANN) to very high-dimansional (DNN) cost space.

The cost function typically has a highly nonlinear dependence on the weights and bias parameters. Due to the **non-convex** nature of the cost function, in practice, there will be many points in the weight space at which the gradient vanishes (or is numerically very small). 

Example of such places are, using three-dimensional metaphors: **local minima, saddle point and plateau**. In these areas, the gradient on every dimension (direction) "vanishes" and the weight update becomes extremely slow. In the worst case, the model stops training and returns the "optimized" weights. Local minima mostly occur in low-dimensional weight spaces, while saddle point and plateau plague high-dimensional cost spaces.

<img src="http://engineering.unl.edu/images/uploads/LocalMinimum.png" width=600 height=300>

The **varying slopes** of the cost function could also delay the training.

There are some regions resembling **cliffs** where the cost space experiences sharp nonlinearity. It could overshoot SGD and diverge learning.

In high-dimensional cost space there is yet another topological factor that could delay the convergence significantly. This factor is observed in narrow ravine-like regions, known as **pathological curvature**. Well, another metaphor!

Below we describe these topological regions that slow down the SGD convergence.


## Local Minima

In the cost function space the global minimum is a point that satisfies the following three requirements.
- The cost is zero
- The gradient is zero
- The Hessian is zero
- Away from the point slope is positive in all dimensions (positive curvature)

The local minima, however, satisfiy the last three requirements. It means that the cost at a local minimum could be large (shallow minimum), or could be low (deep minimum), but usually never zero.

SGD cost function may exhibit many local minima, as shown below. There are lots of very shallow local minima, some deeper ones and a global one. During training there is a risk of getting stuck in a local minimum, which is bad.

Interestingly DNNs are mostly immune to this risk. Because local minima poses a substantial demand on the topology of the cost function: the slope along all dimensions out of the local minima must be positive (i.e., should bend upward due to positive curvature). The probability of this happening, for example in a 10,000 dimensional cost space, is $2^{-10000}$, which is extremely small. Following empirical investigation suggests that local minima are not a problem in DNNs. They may exist but in practice they are hardly ever encountered during SGD. 

https://arxiv.org/abs/1412.6544

Thus, local minima usually exist in lower-dimensional weight space (shallow ANNs). 



<img src="http://engineering.unl.edu/images/uploads/SGD-LocalMinima.png" width=600 height=300>


#### How do we escape local minima?

The stochastic nature of SGD helps to escape local minima. However, the likelihood of escaping from a local minimum depends on how "deep" the valley is. Learning rate plays a key role to avoid local minima. Smaller learning rates make it difficult to escape from local minima.

    
In DNNs, zero/near-zero gradient points are found mostly on the saddle regions or plateaus. The problem is that the first-order algorithms, such as SGD, may get stuck at saddle points and become painfully slow at plateaus.



### Saddle Point


DNN cost space is proliferated with saddle points. In higher-dimensional spaces, local minima are rare and saddle points are more common. For a function $J: \R^d \rightarrow \R$ of this type, the expected ratio of the number of saddle points to local minima grows exponentially with $d$, i.e., number of weight parameters. 


"Saddle point" obviously is a metaphor. We loosely use the shape of a horse saddle to describe a problemetic spot on the cost space.

In mathematics, a saddle point (or minimax point) is a point on the surface of the graph of a function where the slopes (derivatives) in orthogonal directions are all zero, but which is not a local extremum (neither minimum nor maximum) of the function. In ANNs the saddle point is a specific point in the weight space where all components of the gradients are zero. If SGD reaches to a saddle point, it would mistake this point to be the minimum as the gradients of the cost function in all dimension are zero. Thus, there will be no more weight updates, and training stops.

To understand why a saddle point has zero gradient in all dimension, consider the following two-dimensional cost function:

$f(x,y) = x^²- y^²$ 

Let's compute its partial derivatives: 
$\frac{df}{dx} = 2x$
$\frac{df}{dy} = -2y$

Thus, the gradient of this function is [2x, -2y]. At the point (0,0), both components of the gradient are equal to zero. When SGD arrives at a point like this, it stops updating.

<img src="http://engineering.unl.edu/images/uploads/SGD-SaddlePoint.png" width=600 height=300>


Although a saddle point is a zero-gradient point, just like local minima, it differs significantly from local minima. Because around the saddle point not all gradients have positive slope, as shown in the above figure. It is a place where not all gradients of each dimension is towards the same direction. At the saddle point region some dimensions would have positive slope (bend upward), while some dimensions have negative slope (bend downward). Intuitively, this means that a saddle point acts as both a local minima for some neighbors and a local maxima for the others, as shown in the following figure. See the "saddle point" region looks like a horse saddle. But this is as far as metaphor could go. We need a rigorous technique to identify a saddle ponint so that we could escape it.



<img src="http://engineering.unl.edu/images/uploads/SaddlePoint.png" width=800 height=400>


#### How do we mathematically distinguish a saddle point from a local minimum?

To do this we need to consider the second-order derivative $\nabla^2 f(\vec{x})$. It is an $n x n$ matrix, known as the **Hessian**. It's $i, j$-th entry is equal to $\frac{\partial^2}{\partial x_i \partial y_i}f(\vec{x})$. Using the Hessian we could distinguish a saddle point from a local minimum.

- The point $\vec{x}$ is a local minimum: When the Hessian has only positive eigenvalues, i.e., is positive definite (i.e., $u^{\top}\nabla^2f(\vec{x})u  > 0$ for any $u \neq 0$.

- The point $\vec{x}$ is a saddle point: When the Hessian has both positive and negative eigenvalues.


While the gradient of a cost function providesonly unidirectional slope around a point, Hessian  provides the **curvature** around that point. The curvature tells how much the surface deviates from a plane.


For example, in the above example of the saddle point by using the curvature we could figure out that the surface goes down along the y axis and goes up along the x axis. However, for a minimum point (e.g., local minimum) the curvature goes up both along x axis and y axis.


Let's use an example to illustrate this point. We will use two cost functions.

- Cost function 1 $f(x,y) = x^²+ y^2$: the minimum point exists at (0, 0)
- Cost function 2 $f(x,y) = x^²-y^2$: the saddle point exists at (0, 0)


We compute the Hessian of the first cost function: $f(x,y) = x^²+ y^2$

First-order partial derivatives:
$\frac{\partial f}{\partial x} = 2x$
$\frac{\partial f}{\partial y} = 2y$


Second-order partial derivatives:
$\frac{\partial^2 f}{\partial x^2} = 2, \frac{\partial^2 f}{\partial x \partial x} = 0$
$\frac{\partial^2 f}{\partial y \partial x} = 0, \frac{\partial^2 f}{\partial y^2} = 2$

Thus, the Hessian of this function is:

 $
  H =
  \left[ {\begin{array}{cc}
   2 & 0 \\
   0 & 2 \\
  \end{array} } \right]
$

This matrix has a single positive eigenvalue 2. Thus, it's a positive definite matrix.

By multiplying this matrix with unit vectors along the x and y axes (one unit away from the minimum point), we can find out what the curvature looks like:
H * [0, 1] = [0 2] = 2*[0, 1]
H * [0, -1] = [0 -2] = 2*[0, -1]
H * [1, 0] = [2 0] = 2*[1, 0]
H * [ -1, 0] = [-2 0] = 2*[-1, 0]


We see that multiplying H by a unit vector along both x axis and y axis yield a positive multiple of the vector, i.e., the curvature is positive. It means that as we move away from (0, 0) we can only go up. Thus, (0,0) is a minimum along both the x axis and y axis.


Now let's compute the Hessian of the second cost function: $f(x,y) = x^²-y^2$


First-order partial derivatives:
$\frac{\partial f}{\partial x} = 2x$
$\frac{\partial f}{\partial y} = -2y$


Second-order partial derivatives:
$\frac{\partial^2 f}{\partial x^2} = 2, \frac{\partial^2 f}{\partial x \partial x} = 0$
$\frac{\partial^2 f}{\partial y \partial x} = 0, \frac{\partial^2 f}{\partial y^2} = -2$

Thus, the Hessian of this function is:

$
  H =
  \left[ {\begin{array}{cc}
   2 & 0 \\
   0 & -2 \\
  \end{array} } \right]
$

This matrix has two eigenvalues: 2 and -2. 

By multiplying this matrix with unit vectors along the x and y axes, we can find out what the curvature looks like:
H * [0, 1] = [0 -2] = -2*[0, 1]
H * [0, -1] = [0 2] = -2*[0, -1]
H * [1, 0] = [2 0] = 2*[1, 0]
H * [ -1, 0] = [-2 0] = 2*[-1, 0]


We see that multiplying H by a unit vector along x axis yields a positive multiple of the vector, i.e., the curvature along x axis is positive. It means that as we move away from (0, 0) along x axis, we can only go up. Thus, (0,0) is a minimum along x axis.

On the contrary, multiplying H by a unit vector along y axis yields a negative multiple of the vector, i.e., the curvature along y axis is negative. It means that as we move away from (0, 0) along y axis, we can only go down. Thus, (0,0) is a maximum along y axis.

The above analysis suggests that (0, 0) is a saddle point.

In conclusion, by computing the Hessian of the cost function we can find a saddle point, and escape it by moving along the direction of the negative eigenvalue.



However, there are two problems with the Hessian based technique for escaping saddle points.

- Computing the Hessian is quite expensive (requires more memory). We may use other techniques (given below) that are based on only first-order derivative of the cost function.
- Some saddle points have zero curvature.


##### Saddle Point with Zero Curvature

The saddle point that we have discussed above is a **well-behaved** saddle point, and using curvature at the saddle point we can escape it. Intuitively, a saddle point $\vec{x}$ is well-behaved, if there is a direction $u$ such that the second order term $u^{top}\nabla^2f(\vec{x}u$ is significantly smaller than 0. Geometrically this means there is a steep direction where the function value decreases. This type of saddle points are also known as  **strict saddle** points.


The notion of strict saddle point can be modeled via a strict saddle function. A function f(x) is strict saddle if all points $\vec{x}$ satisfy at least one of the following
- Gradient $\nabla f(\vec{x})$ is large.
- Hessian $\nabla^2f(\vec{x})$ has a negative eigenvalue that is bounded away from 0.
- Point \vec{x} is near a local minimum.


Using Hessian (second-order derivative) or some simple algorithms discussed later (based on first-order derivative) we can handle strict saddle points. But, non-convex cost functions can have much more complicated landscapes that involve **degenerate saddle points**. 

Such degenerate structure often indicates a complicated saddle point:
- Figure (a): a monkey saddle
- Figure (b) & (c): a set of connected saddle points 

<img src="http://engineering.unl.edu/images/uploads/SGD-SaddlePoint-Degenerate.png" width=800 height=400>

This type of saddle points has the following characteristics: Hessian is positive semidefinite and have 0 eigenvalues. 

Let's try to understand why Hessian doesn't help to escape a degenerate saddle point. We will use the monkey saddle point for this illustration. We will show that a monkey saddle point has zero curvature.

Let's consider the following function that contains a monkey saddle point at (0, 0).

- $f(x, y) = x^3 - 3xy^2$

Below we show the plot of this function. The saddle point is at (0, 0). The problem is that at (0, 0) the curvature is also zero. Let's see how.


<img src="http://engineering.unl.edu/images/uploads/SGD-SaddlePoint-Monkey.png" width=800 height=400>


We compute the Hessian of the cost function: $f(x,y) = x^3-3xy^2$


First-order partial derivatives:
$\frac{\partial f}{\partial x} = 3x^2 - 3y^2$
$\frac{\partial f}{\partial y} = -6xy$


Second-order partial derivatives:
$\frac{\partial^2 f}{\partial x^2} = 6x, \frac{\partial^2 f}{\partial x \partial x} = -6y$
$\frac{\partial^2 f}{\partial y \partial x} = -6y, \frac{\partial^2 f}{\partial y^2} = -6x$

Thus, the Hessian of this function is:

$
  H =
  \left[ {\begin{array}{cc}
   6x & -6y \\
   -6y & -6x \\
  \end{array} } \right]
$

Observe that at (0, 0) the Hessian is is a zero matrix! Multiplying it by a unit vector (or any vector) will result in a zero vector. Thus, we are unable to figure out the curvature. Neither the gradient nor the Hessian provide any information on which way is down. 

Thus, using Hessian we are unable to escape degenerate saddle points.



#### How do we escape strict saddle points?

There are various techniques to escape strict saddle points.
- Second-order partial derivative (Hessian)
- Noisy SGD
- Random Initialization

We discuss these technique below.

- Second-order partial derivative (Hessian)
We have already discussed how to escape strict saddle points using Hessian. However, second-order SGD is not feasible in DNNs (due to cost and requirement for larger batch size).


- Noisy SGD
For SGD to rapidly avoid saddle points, it needs to have sufficiently high stochastic variance along all directions. A noisy version of SGD is proposed in which when updating parameters, adding a tiny amount of noise on top of the gradient is enough to nudge SGD to a slightly lower point instead of getting stuck at the saddle point. The inherent noise helps in convergence because it pushes the current point away from saddle points.

https://arxiv.org/abs/1503.02101


- Random Initialization
Another solution is that instead of adding random noise we should perform random initialization of weights. It allows SGD to evade first-order saddle points.

https://arxiv.org/abs/1602.04915


Finally, how do we escape degenerate saddle points?

We discussed earler that many functions have degenerate saddle points such that the first and second order derivatives cannot distinguish them with local optima. To escape these saddle points, sadly, we have to use higher-order derivatives.
https://arxiv.org/abs/1503.02101




 
### Plateau 

We use the metaphor of "plateau" to describe wide regions in higher-dimensional cost space at which the gradient (as well as the Hessian) at each dimension is zero or near zero. There is hardly any slope. The consequence would be near-zero updates for all weights, which in turn would mean that we would barely move towards the minimum. We would be stuck on that plateau for a long time and training would be extremely slow. 


<img src="http://engineering.unl.edu/images/uploads/Plateau.png" width=400 height=200>



#### How do we move faster on a plateau?

In DNNs a constant scalar learning rate will take a long time on the plateau to converge. There are two types of solutions that could be used to expedite the movement of SGD on plateau.
- Augment SGD with momentum
- Use adptive learning rate per dimension, such that learning rate increases in regions where cost function slope is flat (i.e., on a plateau)





### Varying Slopes

As we discussed above, a plateau is a wide region on which gradients on all dimension is zero or nearly zero. At the other extreme , there are regions on the cost space that has steeper slopes. In general, in DNN cost functions some dimensions are flatter where the movement would be slower, while some dimensions are steeper where movement would be faster. A constant scalar learning rate, as used by SGD, would surely cause uneven progress and slow down the training process.  

##### How do we expedite training across varying slopes?

We need to adapt the learning rate per dimension. Following optimization algorithms are used for this purpose.
- AdaGrad
- Adadelta
- RMSProp
- Adam
- Nadam
- AdaMax



## Cliff


In some type of DNNs (e.g., in Recurrent Neural Networks) extremely steep regions in the cost space exist. Usig metaphor, these are "cliffs", as shown in the following two-dimensional cost space.

This type of sharp nonlinearity in cost space may arise due to the multiplication of several layers of weights. These nonlinearities give rise to very high derivatives in some places. When the weight parameters get close to such a cliff region, a gradient descent update can catapult the parameters very far even with a modest step size.


<img src="http://engineering.unl.edu/images/uploads/SGD-Cliff.png" width=600 height=300>


Standard solution for avoiding cliffs is to perform gradient clipping. It reduces the step size to be small enough that it is less likely to go outside the region where the gradient indicates the direction of approximately steepest descent.



## Pathological Curvature

The pathological curvature is best explained by the methaphor of a **ravine**, as shown in the following figure. To get down to the minimum of the cost space, we have to go through the ravine. 

<img src="http://engineering.unl.edu/images/uploads/PathologicalCurvature.png" width=600 height=300>

For the convenience of understanding consider a two-dimensional scenario in the following figure. In the ravine, the cost surface at the ridge curves much more steeply in the direction of w1. Thus, SGD would oscillate along the ridges of the ravine, and moving a lot slower towards the minimum. It will require many iterations to converge.


<img src="http://engineering.unl.edu/images/uploads/PathologicalCurvaturePath.png" width=600 height=300>

Let's understand why the movement along w2 is slower. Consider a point on the surface of the ridge. The gradient at the point can be decomposed into two components, one along direction w1 and other along w2. The component of the gradient in direction of w1 is much larger because of the curvature of the cost function, and hence the direction of the gradient is much more towards w1, and not towards w2 (along which the minima lies). 

Thus, the movement towards the minimum oscillates, which requires a larger number of iterations before convergence.

## Augmenting SGD: Faster Optimizers

Generally training DNNs is an excrutiatingly slow process for various reasons. One reason for sure is the regular SGD optimization algorithm. We can overcome the limitations of the vanilla SGD as well as the topological causes of the cost function by using faster optimizers. These optimizers are created by augmenting SGD.


Below we describe the following optimizers: 
- Momentum optimization
- Nesterov Accelerated Gradient 
- AdaGrad
- Adadelta
- Adam
- Nadam
- Adamax



### Momentum Optimization

The Momentum optimization is a small hack of SGD that could reduce its oscillations and accelerate its movement. Previously we have seen that SGD oscillations occur due to two reasons: 

- The stochastic nature of SGD, which gets worse near the minima delaying convergence.
- The topology of the cost space, e.g., varying slopes in pathological curvature that results into poor conditioning of the Hessian (stark difference in the slopes along different directions. 

Adding momentum to SGD can combat both.

For motivating the discussion of Momentum optimization, let's illustrate how the topology of the cost space begets oscillations by using a simple two-dimensional scenario. In the following figure we see that the cost function looks like an elongated bowl and the minimum is located at its center. This region resembles a narrow valley, or pathological curvature. The slope varies gently along the direction of $w_2$, while it varies heavily along the direction of $w_1$. As a consequence as SGD starts moving from the left towards right, it keeps on bouncing back and forth with high amplitude. This happens because of the mismatch in the slope (and curvature) of the the two dimensions. Due to its oscillatory movement, convergence to the minimum becomes slower. 


<img src="http://engineering.unl.edu/images/uploads/SGD-Oscillation.png" width=600 height=300>


We can dampen the oscillations by computing a moving/local average of the gradient (of the cost function). The technique to compute this is known as **Exponentially Weighted Moving Average (EWMA)**.

The EWMA of oscillatory observations (e.g., gradients in SGD) at time $t$ is computed by using two factors:
- The recent past obsevations and 
- The current observation at time $t$ 

To impose the "smoothing", the EWMA puts more weight on the past observations. Using a metaphor, past observations build up "momentum" that stabilizes the fluctuations of the current observation.

- Moving_Average(t) = weight * Past_Observations + (1 - weight) * Observation(t)

Past observations are weighted heavily by the weight parameter (denoted by $\beta$). It determines how far we go into the past to determine the present. In other words, larger $\beta$ will use more observations from past to create increased momentum that squashes out oscillations at present. EWMA is dicussed in detail in the following notebook:



Recall that SGD updates the weights at time $t$ $\vec{w}_t$ by directly subtracting the gradient of the cost function ($\nabla_w J(\vec{w}_t)$) that is multiplied by the learning rate $\eta$:

$\vec{w}_{t+1} = \vec{w}_t - \eta \nabla_w J(\vec{w}_t)$

It does not care about what the earlier gradients were. 

        If the local gradient is tiny, SGD goes very slowly.
        
        
The Momentum optimization not only uses the gradient of the current step to guide the search, but also it accumulates the gradient of the past steps to determine the direction to go. 

In Momentum optimization, for each weight $\vec{w}$, we keep an additional value $\vec{m}$ (it is called the momentum vector). Initially, $\vec{m}$ is set to zero, then it keeps a moving average of the past gradients, and that is how much we update. Following is the Momentum optimization update rule.


1. $\vec{m}_t = \beta \vec{m}_{t-1} + (1 - \beta) \nabla_w J(\vec{w}_t)$
2. $\vec{w}_{t+1} = \vec{w}_{t} -\eta \vec{m}_t$


Here, the momentum vector $\vec{m}_t$ is the retained gradient from previous iterations. To **simulate some sort of friction mechanism** and prevent the momentum from growing too large, the algorithm introduces a new hyperparameter $\beta$, which must be set between 0 (high friction) and 1 (no friction). A typical value of $\beta$ is 0.9.

When $\beta = 0$, we recover SGD. But for $\beta = 0.9$ (or larger, much closer to 1), this appears to be the boost we need. Our iterations regain that speed and boldness it lost, speeding to the optimum with a renewed energy.


Observe that in Momentum optimization, the previous gradients are also included in subsequent updates. For example, if we set the initial value for $\vec{m}$ to 0 and choose our $\beta$ as 0.9, the subsequent update equations would look like (here $g = \nabla_w J(\vec{w})$):

$\vec{m_1} = - g_1$

$\vec{m_2} = - 0.9*g_1 - g_2$

$\vec{m_3} = - 0.9(0.9*g_1 - g_2) - g_3 = -0.81*g_1 - 0.9*g_2 - g_3$


Thus, although the previous gradients are also included in subsequent updates, the weights of the most recent gradients are more than the less recent ones. 

 
 
### How does Momentum Optimiztion dampen oscillations?

Let's see how momentum dampens the oscillations in the above figure. First, we state the momentum update equations for both $w_1$ (flatter dimension) and $w_2$ (steeper dimension)


- $\vec{m_{w1}}_t = \beta \vec{m_{w_1}}_{t-1} + (1 - \beta) \nabla_{w_1} J(\vec{w_1}_t)$
- $\vec{m_{w2}}_t = \beta \vec{m_{w_2}}_{t-1} + (1 - \beta) \nabla_{w_2} J(\vec{w_2}_t)$



If we average out the gradients for both $w_1$ and $w_2$, we find that the oscillations in the steeper direction will tend to average out to something closer to zero. Thus, in the steeper direction, where we want to slow things down, this will average out positive and negative numbers. The average will be close to zero. Whereas, on the flatter direction, all the derivatives are pointing to the right. Hence the average in the flatter direction will still be pretty big. That's why within a few iterations we find that the SGD with momentum ends up eventually just taking steps that are much smaller oscillations in the steeper direction, but are more directed to just moving quickly in the flatter direction. This allows Momentum optimization to take a more straightforward path by dampening out the oscillations in this path to the minimum. 


### Metaphor for Momentum Optimization
Using a physical metaphor we could understand how momentum expedites the movement. Imagine that we are dropping a symmetric ball from one corner of the elongated bowl in the above figure. According to Newton's second law of motion, the ball would experience a downward force to move towards the bottom of the bowl. This force is equal to its mass times acceleration. In our example, assume that the ball has unit mass. Thus, it gets acceleration from the negative gradient term. 

On the other hand, the ball acquires some momentum, which is the product of its mass and velocity. Assuming unit mass, the velocity of the ball can be regarded as its momentum.

Thus, the gradients provide acceleration to the ball, while momentum imparts the velocity. Unlike in SGD that takes every single step independently of all previous steps, as the ball rolls downhill it gains momentum by accumulating past gradients.

A pertinent question is: if the ball is dragged downward by the acceleration (i.e., negative of its gradient), why do we need to add momentum? The acceleration should be enough to roll the ball, right?

We need to add the second force (the momentum) to apply a "viscous drag". Otherwise the ball, being dragged by acceleration alone, would oscillate forever due to varying gradients. This causes the ball to gradually lose energy over time and after a long time to converge to a local minimum. The momentum, which is the viscous drag, is need to tame the oscillation. It is weak enough so that the gradient can continue to cause motion until a minimum is reached, but strong enough to prevent motion if the gradient does not justify moving.


Bofore ending the dicussion on Momentum optimization, let's give an alternative version of its update rule.

1. $\vec{m}_t = \beta \vec{m}_{t-1} - \eta \nabla_w J(\vec{w}_t)$
2. $\vec{w}_{t+1} = \vec{w}_t + \vec{m}_t$


This is a less intuitive version of the Momentum optimization update rule, which some people prefer to use. It is essentially the same as the former one. We just need to re-estimate $\eta$ for dropping the $(1 - \beta)$ term before the gradient. In practice, both versions work just fine, except the latter affects the best value of $\eta$. 




### Nesterov Accelerated Gradient

    Can we improve Momentum optimization to further dampen oscillations? 

To describe Momentum optimization we used the metaphor of a symmetric ball rolling downward from the corner of an elongated bowl shaped cost space. The ball rolled downward through a reasonably straight path due to the momentum that it accrued from the past gradients. During its journey sometimes the ball needs to slow-down/speed-up whenever the **local slope** varies along the path. This might deviate the ball from its course temporarily. Thus, there would still be some oscillations, even after applying momentum. How do we further dampen this oscillation?


One idea is that we could tell the ball not to choose the direction of movement based on the current location (which it usually does by computing the gradient at time $t$ $\vec{w}_t$). Instead we could just look at the momentum vector at time $(t-1)$ ($\vec{m}_{t-1}$) to see where it leads to at the next time step at $t$. We know that the momentum vector will follow the optimal path to the minimum of the cost function. Thus there is an assurance that at time $t$ it will reach an optimal point along its best curse. We can use it anticipated future location at time $t$ to decide the direction of the next step for the ball (by cmputing the gradient). Thus, instead of computing the gradient at the current location $\vec{w}_t$ in the cost space at $t$, we could ask the ball to jump ahead into the direction of the momentum vector, and from that **new location** ($\vec{w}_t + \beta \vec{m}_{t-1}$) to choose the direction of the next step (by computing the gradient at the new location). This way we could create a **smarter ball** that moves based on the prediction of its future optimal path.

In short, the smart ball doesn't make decisions at each step based on its present, instead takes a blind and small leap based on where future could lead to. Thus, it's motion, being guided by near future, should reduce the oscillation along its trajectory. And, indeed, it does so! 

Nesterov Accelerated Gradient (NAG) method, also known as Nesterov momentum optimization, does exactly this. It was proposed by Yurii Nesterov in 1983. It tweaks the Momentum optimization update rule a bit by incorporating the future path information. The difference between NAG and standard Momentum is where the gradient is evaluated. 

To understand intuitively how NAG works, let's consider the following two-dimensional cost space.
- Blue arrow: represents the regular Momentum update
- Brown + red arrow: represents the NAG update
- Green arrow: represents the final direction to which both regular Momentum and NAG would lead to (eventually)

Observe that Momentum first computes the current gradient (small blue vector) at the local position $\vec{w}_t$ at $t$. Then, it takes a big jump in the direction of the updated accumulated gradient (big blue vector). Thus, it deviates further from the destination (tip of the first green arrow). Thus, due to this deviation, its movement towards the final destination will be oscillatory.


On the other hand, NAG does not compute the gradient at the local position $\vec{w}_t$ at $t$, but slightly ahead in the direction of the momentum, at $\vec{w}_t + \beta \vec{m}_{t-1}$. In other words, first it makes a big jump in the direction of the previous accumulated gradient (brown vector). Then, it measures the gradient and moves towards the direction of gradient (red vector). It results in the complete NAG update (green vector). This anticipatory update dampens the oscillation and expedites the movement.


<img src="http://engineering.unl.edu/images/uploads/SGD-NesterovMomentum1.png" width=600 height=300>

Following is the NAG update rule.

1. $\vec{m}_t = \beta \vec{m}_{t-1} + (1 - \beta) \nabla_w J(\vec{w}_t + \beta \vec{m}_{t-1})$
2. $\vec{w}_{t+1} = \vec{w}_t -\eta \vec{m}_t$


This small tweak works because in general the momentum vector will be pointing in the right direction (i.e., towards the minimum), so it will be slightly more accurate to use the gradient measured a bit farther in that direction rather than the gradient at the original position.

To further explain how NAG improves Momentum optimization, consider the following figure. We can see $\nabla_1$ represents the gradient of the cost function measured at the starting point at t $\vec{w}_t$, and $\nabla_2$ represents the gradient at the point located at $\vec{w}_t + \beta \vec{m}_{t-1}$.


<img src="http://engineering.unl.edu/images/uploads/SGD-NesterovMomentum2.png" width=600 height=300>

Observe that the NAG update ends up slightly closer to the optimum. After a while, these small improvements add up and NAG ends up being significantly faster than regular Momentum optimization.

Moreover, note that when the Momentum pushes the weights across a valley, $\nabla_1$ continues to push farther across the valley, while $\nabla_2$ pushes back toward the bottom of the valley. This helps reduce oscillations and thus NAG converges faster.


## Beyond Momentum Optimization and NAG

Momentum optimization and NAG use a scalar constant learning rate for all dimension. They augment SGD by adapting the weight vector using past gradients. 

However, some problems require adapting learning rates **per dimension**. AdaGrad, as well as the remaining optimization algorithms, are designed to do this. These algorithms belong to the class of optimizers, sometimes known as "fast optimizers", in contrast to constant scalar learning rate based SGD.

These fast optimizers draw their inspiration from second-order SGD that in which the update is based on the Newton method:

$\vec{w}_{t+1} = \vec{w}_t - \eta H^{-1} \odot g $

where $\eta$ is often one or close to one (damped-Newton). The notation $\odot$ is used to denote the matrix-vector product.

A computationally feasible approxmation of the Newton method uses the diagonal approximation of the Hessian, instead of the full Hessian:

$\vec{w}_{t+1} = \vec{w}_t - \frac{\eta}{diag(H)} \odot g $


The Newton method, which is computationally expensive and are not used in practice, provides a very useful insight to define the following general rule for optimization that is faster than a scalar constant learning rate based SGD.

Find any preconditioner matrix $B$ that improves the performance of SGD, but without involving expensive computation ($\approx O(d)$) of the preconditioner:

$\vec{w}_{t+1} = \vec{w}_t - \frac{\eta}{B} \odot g $

Here, the diagonal matrix $B$ is used to scale the learning rate $\eta$ per dimension.

We will see that the fast optimizers compute the preconditioner matrix $B$ in such a fashion that it prodives some approximation of the diagonal approximation of Hessian.


## AdaGrad

Before formally presenting the AdaGrad algorithm, let's motivate the need for updating learning rate per dimension.

In training DNNs, the contributions of the weights to the error/cost are not the same. Thus, we should adapt the learning rate separately for each weight, depending on the convergence along that direction. In directions where the gradient is small (e.g., flatter slope) the learning rate should be larger to compensate, and it should be smaller along directions where the gradient is already larger (i.e., steeper slope).

This is accomplished by the Adaptive Gradient (AdaGrad) algorithm, which was proposed in 2011.
http://jmlr.org/papers/volume12/duchi11a/duchi11a.pdf


AdaGrad dynamically incorporates knowledge of the geometry of the data observed in earlier iterations to perform more informative gradient-based learning. It adapts the learning rate to the individual weights. It performs: 
- Smaller updates (i.e., apply small learning rate) for weights associated with frequently occurring features
- Larger updates (i.e., apply large learning rate) for weights associated with infrequent features 

For this reason, it is well-suited for dealing with **sparse data**. For example, in word embeddings problem many words would be infrequent, thus their weights would require larger updates as compared to the frequent words. The GloVe word embeddings was trained using AdaGrad. 
https://www.aclweb.org/anthology/D14-1162.pdf



Let's present the AdaGrad algorithm.

For scaling $\eta$ per dimesion, AdaGrad defines the preconditioner matrix by taking the outer product of the $d$-dimensional gradient vector $\nabla J(\vec{w})$ with itself and creates a $d x d$ diagonal matrix. In this matrix, each diagonal element $(i, i)$ is the square of the $i$-th partial derivative $\frac{\partial J(\vec{w})}{\partial w_i}$:

$B \equiv diag(\nabla J(\vec{w} \otimes \nabla J(\vec{w})$

Here, $\otimes$ denotes outer product or element-wise multipliation of two gradient vectors.

Using this diagonal preconditioner matrix $B$, the AdaGrad update rule is:

1. $\vec{s}_t = \vec{s}_{t-1} + B_t$
2. $\vec{w}_{t+1} = \vec{w}_t - (\eta \oslash \sqrt{\vec{s}_t + \epsilon}) \odot \nabla_w J(\vec{w}_t)$

Observe that this algorithm relies only on first-order information, but has some properties of second-order methods via $\vec{s}$. It also has annealing effect. Let's parse the rule. 

The first step computes the accumulated square of the gradients. Note that $\vec{s}$ is a diagonal matrix $\R^{d x d}$ where each diagonal element $(i, i)$ is the sum of squares of the partial derivative with respect to $w_i$ up to the current time-step. To compute the squares of the gradients we perform element-wise multiplication (denoted by $\otimes$). Thus, for each weight $w_i$, it computes $s_i$, which is the the accumulated square of the partial derivatives of the cost function $J(\vec{w})$ with respect to $w_i$:

$s_i \leftarrow s_i + (\frac{\partial J(\vec{w})}{\partial w_i})^2$

If the cost function is steep along the $i$-th dimension, then $s_i$ will keep on getting larger at each iteration.


In the second step, on the right-hand side a global learning rate $\eta$ is shared by all dimensions. However, it is scaled per dimension, being divided by the $l_2$ norm of all previous gradients on a per-dimension basis. This step updates the weights by subtracting the matrix-vector product (denoted by $\odot)$ of the scaled learning rate and the gradient. The learning rate is scaled down by a factor of $\sqrt{\vec{s} + \epsilon}$. Here $\epsilon$ is used as a smoothing term that avoids division by zero. Usually its value is set to $1e^{-8}$. For a single weight $w_i$ this update looks like folowing:

$w_i \leftarrow w_i -\frac{\eta}{\sqrt{s_i + \epsilon}} \frac{\partial J(\vec{w})}{\partial w_i}$

Notice that the learning rate decays as iteration increases. However, the decay is not same for all weights.
- The decay happens faster for steeper dimensions
- The decay happens slowly for flatter dimensions

There are at least three benefits of adaptive learning in AdaGrad:
- The updates lead towards the minimum faster in regions of varying slopes, as shown in the following figure.
- Due to accumulation of gradients in the denominator of the update rule, it has the same effect as annealing, i.e., reducing the learning rate over time.
- It requires much less tuning (or no tuning) of the learning rate hyperparameter $\eta$. Most implementations use a default value of 0.01.


<img src="http://engineering.unl.edu/images/uploads/SGD-AdaGrad.png" width=600 height=300>


### Beyond AdaGrad

AdaGrad is designed to converge rapidly when applied to a convex function, but it often stops too early when training non-convex functions in ANNs. It is not advised to use in training DNNs. 

We give three limitations of AdaGrad:

- We wtill need to manually select a global learning rate.

- It can be sensitive to initial conditions of the parameters and the corresponding gradients. Because the magnitudes of gradients are factored out. If the initial gradients are large, the learning rates will be low for the remainder of training. This can be combatted by increasing the global learning rate, which makes it sensitive to the choice of learning rate. 

- Its main weakness is that the learning rate gets scaled down so much that the algorithm ends up stopping entirely before reaching the global optimum. This happens due to its accumulation of the squared gradients in the denominator. Since every added term is positive, the accumulated sum keeps growing during training. This in turn causes the learning rate to shrink and eventually become infinitesimally small, at which point the algorithm is no longer able to acquire additional knowledge. It simply stops the training prematurely!

These limitations are addressed by the AdaDelta algorithm.


## Adadelta


Adadelta is an extension of AdaGrad that seeks to improve upon the two main drawbacks of AdaGrad:

1. The continual decay of learning rates throughout training.
2. The need for a manually selected global learning rate. 


Instead of accumulating all past squared gradients, in Adadelta the sum of gradients is recursively defined as an exponentially decaying average of all past squared gradients. This is exactly similar to what we did in Momentum optimization to reduce gradient oscillations by using the EWMA technique. In Adadelta, the moving average $E[\nabla_w J(\vec{w})^2]_t$ at the current time step $t$ then depends (as a fraction $\gamma$ similarly to the Momentum term) only on the previous average and the current gradient:


$E[\nabla_w J(\vec{w})^2]_t = \gamma E[\nabla_w J(\vec{w})^2]_{t-1} + (1 - \gamma) \nabla_w J(\vec{w}_t)^2$


Then, during the weight update, the learning rate parameter $\eta$ is scaled down by factor of the decaying average over past squared gradients $E[\nabla_w J(\vec{w})^2]$:

$\vec{w}_{t+1} = \vec{w}_t - \frac{\eta}{\sqrt{E[\nabla_w J(\vec{w})^2]_t + \epsilon}} \nabla_w J(\vec{w}_t)$


Notice that in the scaling of the learning rate, the denominator is just the root mean squared (RMS) error criterion of the gradient. Using the short-hand RMS, the update rule reads.

$\vec{w}_{t+1} = \vec{w}_t - \frac{\eta}{ RMS[\nabla_w J(\vec{w}_t)]} \nabla_w J(\vec{w}_t)$


So far, we have made only one change to the AdaGrad algorithm, which is, we have replaced the accumulated past gradients in the AdaGrad by an exponentially decaying average of past gradients. This prevents the problem of continual decay of learning rates throughout training.

There is yet another drawback of AdaGrad that Adadelta aspires to solve: use of a manually selected global learning rate. We will see that Adadelta will remove the the global learning rate completely from the update rule.

But is it possible? If so, how?

It appears that Adadelta removes the global learning rate by using a trivial observation and an ingenious analogy.


- Trivial observation: 

We see that the units of the terms in the update rule do not match, i.e., weight parameter unit and the far right term are a mismatch. Adadelta finds it dissatisfactory.


Recall that this issue was also present in SGD, Momentum and AdaGrad algorithms. For example, the units in SGD and Momentum relate to the gradient, not the weight parameter. This can beobserved by considering the update rule of SGD.

$\vec{w}_{t+1} = \vec{w}_t - \eta \nabla_w J(\vec{w}_t)$

On the far right the units of partial derivative would be: $\frac{\partial J(\vec{w})}{\partial w_i} \propto \frac{1}{units\_of\_w_i}$

We see that it doesn't match with the unit of the weight parameter. It was not an issue in SGD, Momentum and AdaGrad. But Adadelta is bothered about this issue. 

The question is how does it fix this issue?

- Ingenious Analogy

It draws inspiration from the Newton method, which, interestingly, doesn't suffer from unit mismatch issue in its update rule.

$\vec{w}_{t+1} = \vec{w}_t - H^{-1}g$

On the far right, the unit of the term $H^{-1}g$ matches with the unit of $w_i$ as follows: 

$H^{-1}g \propto \frac{\frac{\partial J(\vec{w})}{\partial w_i}}{\frac{\partial^2 J(\vec{w})}{\partial w_i^2}} \propto (units\_of\_w_i) $ 


Using an analogy with the Newton method, Adadelta resolves the unit mismatch issue by substituting the global scalar constant learning rate $\eta$ by an expression that has the unit of $\vec{w}$. Adadelta defines this expression by using an exponentially decaying average of squared weight updates:

$E[\vec{w}^2]_t = \gamma E[\vec{w}^2]_{t-1} + (1 - \gamma) \vec{w}_t^2$


The root mean squared error of the weight parameter updates is thus:

$RMS[\vec{w}]_t = \sqrt{E[\vec{w}^2]_t + \epsilon}$


Since $RMS[\vec{w}]_t$ is unknown, we approximate it with the RMS of weight updates until the previous time step. By replacing the learning rate $\eta$ with $RMS[\vec{w}]_{t-1}$ we get the Adadelta update rule:


$\vec{w}_{t+1} = \vec{w}_t -\frac{RMS[\vec{w}]_{t-1}}{RMS[J(\vec{w})]_t}J(\vec{w}_t) $

Here the same constant $\epsilon$ is added to the numerator RMS as well. This constant serves the purpose of both to start off the first iteration where update is zero and to ensure progress continues to be made even if previous updates become small.

Thus we see that by resolving the unit mismatch issue, Adadelta removes the scalar global learning learning rate. Wow!


Adadelta update rule makes the assumption of diagonal curvature so that the second derivatives could easily be rearranged. A consequence is that the squared gradients are aprroximation of the Hessian (more precisely its diagonal approximation). Thus, it uses a pseudo-curvature per dimension to mimic a Newton-like update. But, the good thing is that it costs only one gradient computation per iteration by leveraging information from past updates. 

One limitation of Adadelta is that it does not have an explicit annealing schedule, which might delay convergence near a minimum. For this reason Mometum optimization, that does annealing, is comparatively faster than Adadelta. With momentum, oscillations that can occur near a minima are smoothed out, whereas with Adadelta these can accumulate in the numerator. 



## RMSProp


RMSProp and Adadelta have both been developed independently around the same time stemming from the need to resolve AdaGrad's radically diminishing learning rates. RMSProp, in fact, is identical to the first update vector of Adadelta that we derived previously.


1. $\vec{s}_t = \beta \vec{s}_{t-1} + (1 - \beta) \nabla_w J(\vec{w}_t) \otimes \nabla_w J(\vec{w}_t)$
2. $\vec{w}_{t+1} = \vec{w}_t -(\eta \oslash \sqrt{\vec{s}_t + \epsilon}) \nabla_w J(\vec{w}_t)$

Empirically, RMSProp has been shown to be an effective and practical optimization algorithm for DNNs. It is currently one of the go-to optimization methods being employed routinely by deep learning practitioners.


An interesting fact about RMSProp is that it was actually first proposed not in any academic research paper, but in a Coursera course that Geoffrey Hinton taught on Coursera in 2012.
http://www.cs.toronto.edu/~tijmen/csc321/slides/lecture_slides_lec6.pdf




## Adam


Adam stands for Adaptive Moment Estimation. It combines the ideas of Momentum optimization and RMSProp.

Just like Momentum optimization, it keeps track of an exponentially decaying average of past gradients; and just like RMSProp and Adadelta, it keeps track of an exponentially decaying average of past squared gradients. Whereas Momentum optimization can be seen as a ball running down a slope, Adam behaves like a heavy ball with friction, which thus prefers flat minima in the cost space.

Following are the Adam update equations.

1. $\vec{m}_t = \beta_1 \vec{m}_{t-1} + (1 - \beta_1) \nabla_w J(\vec{w})_t$
2. $\vec{s}_t = \beta_2\vec{s}_{t-1} + (1 - \beta_2) \nabla_w J(\vec{w})_t\otimes \nabla_w J(\vec{w})_t $
3. $\hat{\vec{m}_t} = \frac{\vec{m}_t}{1 - \beta_1^t}$
4. $\hat{\vec{s}_t} = \frac{\vec{s}_t}{1 - \beta_2^t}$
5. $\vec{w}_{t+1} = \vec{w}_t - \frac{\eta} {\sqrt{\hat{\vec{s}_t} + \epsilon}} \hat{\vec{m}_t}$


Let's explain these equations.

- Eq. 1: Computes the exponentially decaying average of the gradient for each weight parameter. It is the estimate of the first moment (the mean). 
- Eq. 2: Computes the exponentially decaying average of the squares of the gradient for each weight parameter. It is the estimate of the second moment (the uncentered variance) of the gradients.

The vectors $\vec{m}_t$ and $\vec{s}_t$ are initialized with zeros. It has been observed that these vectors are biased towards zero, especially during the initial time steps, and especially when the decay rates are small (i.e., $\beta_1$ and $\beta_2$ are close to 1).

Thus, to counteract these biases Adam computes bias-corrected first and second moment estimates in the third and fourth equations.

- Eq. 3: Computes the average of the gradient.
- Eq. 4: Computes the average of square of gradients.

Finally, the bias-corrected vectors $\hat{\vec{m}_t}$ and $\hat{\vec{s}_t}$ are used to update the weight parameters (similar to the update in  Adadelta and RMSProp) giving the the Adam update rule:

- Eq. 5: To decide the learning step, we multiply our learning rate by average of the gradient (Eq. 3) and divide it by the root mean square of the exponential average of square of gradients (Eq. 4). And then we add the update.  

Since Adam is an adaptive learning rate algorithm, it requires less tuning of the learning rate hyperparameter $\eta$. We can often use the default value $\eta = 0.001$, making Adam even easier to use than SGD.


The momentum decay hyperparameter $\beta_1$ is typically initialized to 0.9, while the scaling decay hyperparameter $\beta_2$ is often initialized to 0.999. The smoothing term $\epsilon$ is usually initialized to a tiny number such as $10^{-7}$. 

We end the discussion on the fast optimizers by presenting two variants of Adam:
- Nadam 
- AdaMax


## Nadam


Nadam (Nesterov-accelerated Adaptive Moment Estimation) combines Adam with NAG. Thus, it often converges slightly faster than Adam. In order to incorporate NAG into Adam, we need to modify its momentum term.

First, recall the NAG update rule.

1. $\vec{m}_t = \beta \vec{m}_{t-1} + (1 - \beta) \nabla_w J(\vec{w}_t + \beta \vec{m}_{t-1})$
2. $\vec{w}_{t+1} = \vec{w}_t -\eta \vec{m}_t$

Observe that NAG applies the momentum step twice: one time for updating the gradient (on the far right of the first equation) and a second time for updating weight parameter (in the second equation).

Unlike NAG, Nadam applies the look-ahead momentum vector directly to update the current weight parameters:

1. $\vec{m}_t = \beta \vec{m}_{t-1} + (1 - \beta) \nabla_w J(\vec{w}_t)$
2. $\vec{w}_{t+1} = \vec{w}_t -\eta (\beta \vec{m}_t + (1 - \beta) \nabla_w J(\vec{w}_t))$


Notice that rather than utilizing the previous momentum vector $\vec{m}_{t-1}$ as in the equation of the expanded momentum update rule above, we now use the current momentum vector $\vec{m}_t$ to look ahead. 

In order to add Nesterov momentum to Adam, we can thus similarly replace the previous momentum vector with the current momentum vector in the update rule.

Consider the Adam update rule. We excluded the $\vec{s}$ terms as we don't need to modify it.

$\vec{m}_t = \beta_1 \vec{m}_{t-1} + (1 - \beta_1) \nabla_w J(\vec{w})_t$

$\hat{\vec{m}_t} = \frac{\vec{m}_t}{1 - \beta_1^t}$

$\vec{w}_{t+1} = \vec{w}_t - \frac{\eta} {\sqrt{\hat{\vec{s}_t} + \epsilon}} \hat{\vec{m}_t}$

The update rule can be rewritten as follows:

$\vec{w}_{t+1} = \vec{w}_t - \frac{\eta} {\sqrt{\hat{\vec{s}_t} + \epsilon}} \frac{\vec{m}_t}{1 - \beta_1^t}$

=> $\vec{w}_{t+1} = \vec{w}_t - \frac{\eta} {\sqrt{\hat{\vec{s}_t} + \epsilon}} (\frac{\beta_1 \vec{m}_{t-1}}{1 - \beta_1^t} + \frac{(1-\beta_1)\nabla_w J(\vec{w}_t)}{1-\beta_1^t})$



We see the $\frac{\beta_1 \vec{m}_{t-1}}{1 - \beta_1^t}$ is just the bias-corrected estimate of the momentum vector of the previous time step. We can thus replace it with $\hat{\vec{m}}_{t-1}$:

$\vec{w}_{t+1} = \vec{w}_t - \frac{\eta} {\sqrt{\hat{\vec{s}_t} + \epsilon}} (\beta_1 \hat{\vec{m}}_{t-1} + \frac{(1-\beta_1)\nabla_w J(\vec{w})_t}{1-\beta_1^t})$


We can now add Nesterov momentum just as we did previously by simply replacing this bias-corrected estimate of the momentum vector of the previous time step $\hat{\vec{m}}_{t-1}$ with the bias-corrected estimate of the current momentum vector $\hat{\vec{m}}_t$, which gives us the Nadam update rule:


$\vec{w}_{t+1} = \vec{w}_t - \frac{\eta} {\sqrt{\hat{\vec{s}_t} + \epsilon}} (\beta_1\hat{\vec{m}_t} + \frac{(1-\beta_1) \nabla_w J(\vec{w})_t}{1 - \beta_1^t})$



It has been shown that Nadam generally outperforms Adam but is sometimes outperformed by RMSProp.
https://openreview.net/pdf?id=OM0jvwB8jIp57ZJjtNEZ


## AdaMax

AdaMax was introduced in the same paper as Adam with a small replacement. To understand this first consider the equation for the exponentially decaying average of the squares of the gradient.

$\vec{s}_t = \beta_2\vec{s}_{t-1} + (1 - \beta_2) \mid g_t \mid^2 $

where $g_t = \nabla_w J(\vec{w})_t\otimes \nabla_w J(\vec{w})_t$

This $\vec{s}_t$ factor in the Adam update rule scales the gradient inversely proportionally to the $l_2$ norm of the past gradients. AdaMax replaces the $l_2$ norm with $l_{\infty}$ norm. Generally max norm exhibits stable behavior. 

$\vec{s}_t = \beta_2^{\infty}\vec{s}_{t-1} + (1 - \beta_2^{\infty}) \mid g_t \mid^{\infty} $

$\vec{s}_t = max(\beta_2 \vec{s}_{t-1}, \mid g_t \mid)$

Plugging this into the Adam update rule provides the AdaMax update rule:

$\vec{w}_{t+1} = \vec{w}_t - \frac{\eta} {\vec{s}_t} \hat{\vec{m}_t}$

Note that as $\vec{s}_t$ relies on the max operation. Thus, we don't need to bias $\vec{m}_t$ and $\vec{s}_t$ towards zero. That's why we don't need to compute a bias correction for $\vec{s}_t$. For the hyperparameters we use the same default values as we used in Adam.

Although replacing $l_2$ norm with $l_{\infty}$ norm can generally make AdaMax more stable than Adam, it really depends on the dataset. Thus, we would probably not try AdaMax unless we don't get good performance from Adam.



## Beyond the Horizon of Fast Optimizers ... 

We have seen that all fast optimizers have three characteristics in common:
- They adapt learning rate per dimension by mimicing second-order method. 
- To implement second-order method, they define a pseudo-diagonally-approaximated-Hessian of the cost function.
- They use exponentially decaying moving average of the gradients to define the above second-order term.

Adam brings a bit more improvement by incorporating momentum from SGD Momentum optimization technique.

For training DNNs, it's a standard practice to use either RMSProp, Adam or Nadam. Looks like we have successfully cured the **Achilles' heel** of SGD by augmenting SGD with adaptive learning techniques.

But ironically the Trojan war veteran, great Achilles, regardless of his vulnerability, still conqurs over its faster optimizer cousins. It has been shown that the good, old, first-order SGD (with Momentum optimization) outperforms the standard fast optimizers.
- Object recognition: https://arxiv.org/abs/1608.06993
- Machine translation: https://arxiv.org/abs/1611.04558

Well, this is clearly demotivating; especially after all the eulogies that we have given so far to the second-order methods. We are not content with this observation because honing of SGD to the level of a silver-bullet requires to put a lot of power. It's not more than a brute-force exercise, and sometimes sheer luck. Both are bad for science. There is no proud being Goliath!

The main reason that the fast optimizers don't perform at the level they are touted to be is: may be the pseudo second-order treatments that we have embedded into the fast optimizers are far from being Hessian-like. Perhaphs we need to look beyond the pseudo second-order techniques. Otherwise it is highly unlikely that we would be able to make real progress, at the minimum to beat the Goliath, i.e., the super power infused first-order SGD with Momentum (or some variant of it).

We realize that being Achilles is not so bad, even with his vulnerable heel. It's the Golaith that we most fear about being turned into. So, let's keep the search for David open...

Meanwhile researchers are trying to tilt the fast optimizers to conqur in some domains by improving some or all of the above-methioned characteristics of the adaptive learning techniques. 

For example, AmsGrad identifies the exponential moving average of past squared gradients as a reason for the poor generalization behavior of adaptive learning rate methods. It argues that using some past gradients is tantamount to adding only a "short-term memory" of the gradients, which could delay learning in some scenarios. To alleviate this limitation, AMSGrad uses the maximum of past squared gradients. However, its performance benefit over Adam was observed only in smaller dataset: https://openreview.net/pdf?id=ryQu7f-RZ

There are other optimizers that were proposed in the same vein of improvement.
- AdamW: it fixes weight decay in Adam https://arxiv.org/abs/1711.05101
- QHAdam: it averages a standard SGD step with a momentum SGD step https://arxiv.org/abs/1810.06801
- AggMo: it combines multiple momentum terms https://arxiv.org/abs/1804.00325


These incremental improvements will continue until there is a paradigm shift in algorithmic development. For now it looks like we can only get the taste of the second-order methods via hardware extensions:

https://icml.cc/2011/papers/210_icmlpaper.pdf